In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import math

import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

from model import TDNN

/home/khs1/miniconda3/envs/cw/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device :', device)

Device : cpu


# Data 손보기

In [2]:
import pandas as pd
df_tgt = pd.read_csv('data/target.csv')
df = pd.read_csv('data/data.csv')

/tmp/ipykernel_1677427/1779014282.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/data.csv')


In [3]:
import pandas as pd
df_tgt = pd.read_csv('data/target.csv')
df = pd.read_csv('data/data.csv')

/tmp/ipykernel_1677427/1779014282.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/data.csv')


In [4]:
df_tgt = df_tgt.drop(['Unnamed: 0.1','Unnamed: 0'], axis = 1)
df_tgt

,창녕함안보_수위(EL.m)
0,5.45
1,5.45
2,5.45
3,5.45
4,5.45
...,...
81930,4.89
81931,4.89
81932,4.90
81933,4.90


In [5]:
df = df.drop(['수산대교(창원)_수위'], axis = 1)
df

,Unnamed: 0,여의리(의령군)_수위,의령군(정암교)_수위,함안군(서출리)_수위,합천군(적포교)_수위,길곡_강수량(mm),대병_강수량(mm),창녕_강수량(mm),청덕_강수량(mm),함안_강수량(mm)
0,"('2012-08-27 1:00', '2012-08-27 01:00:00')",0.57,3.83,4.41,0.63,0.0,0.0,0.0,0.0,0.0
1,"('2012-08-27 2:00', '2012-08-27 02:00:00')",0.57,3.82,4.4,0.63,0.0,0.0,0.0,0.0,0.0
2,"('2012-08-27 3:00', '2012-08-27 03:00:00')",0.57,3.81,4.39,0.62,0.0,0.0,0.0,0.0,0.0
3,"('2012-08-27 4:00', '2012-08-27 04:00:00')",0.57,3.79,4.38,0.61,0.0,0.0,0.0,0.0,0.0
4,"('2012-08-27 5:00', '2012-08-27 05:00:00')",0.57,3.77,4.37,0.60,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
81930,"('2021-12-31 19:00', '2021-12-31 19:00:00')",10.09,0.17,1.82,4.36,0.0,0.0,0.0,0.0,0.0
81931,"('2021-12-31 20:00', '2021-12-31 20:00:00')",10.08,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0
81932,"('2021-12-31 21:00', '2021-12-31 21:00:00')",10.09,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0
81933,"('2021-12-31 22:00', '2021-12-31 22:00:00')",10.1,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0


## 결측치('-') 예측

In [6]:
# 결측치('-') 데이터 수 카운트 | 여의리(의령군)_수위 : 1868개 / 함안군(서출리)_수위 : 134개
i = 3
print(df.columns[i])
miss = df[df[df.columns[i]] == '-']
print(len(miss))

함안군(서출리)_수위
134


#### 여의리(의령군)_수위 결측치 예측 (RandomForest)

In [7]:
from sklearn.ensemble import RandomForestRegressor

miss = df[df['여의리(의령군)_수위'] == '-']

drop_df = df.drop(miss.index) # 결측치('-')의 인덱스 제거
drop_df = drop_df.drop(['Unnamed: 0'], axis = 1) # RF에 넣기 위해 날짜값 제거

train_drop_df = drop_df.drop(['여의리(의령군)_수위', '함안군(서출리)_수위'], axis = 1) # label로 쓰일 여의리와 결측치('-')가 있는 함안군 제거
label_drop_df = drop_df['여의리(의령군)_수위'] # label



rf = RandomForestRegressor()
rf.fit(train_drop_df, label_drop_df)

# 결측치 예측
pred = rf.predict(miss.drop(['여의리(의령군)_수위', '함안군(서출리)_수위', 'Unnamed: 0'], axis = 1))
pred


df.loc[miss.index, '여의리(의령군)_수위'] = pred

#### 함안군(서출리)_수위 결측치

In [8]:
miss = df[df['함안군(서출리)_수위'] == '-']

drop_df = df.drop(miss.index) # 결측치('-')의 인덱스 제거
drop_df = drop_df.drop(['Unnamed: 0'], axis = 1) # RF에 넣기 위해 날짜값 제거

train_drop_df = drop_df.drop(['함안군(서출리)_수위'], axis = 1) # label로 쓰일 함안군 제거
label_drop_df = drop_df['함안군(서출리)_수위'] # label



rf = RandomForestRegressor()
rf.fit(train_drop_df, label_drop_df)

# 결측치 예측
pred = rf.predict(miss.drop(['함안군(서출리)_수위', 'Unnamed: 0'], axis = 1))
pred


df.loc[miss.index, '함안군(서출리)_수위'] = pred

In [11]:
print(df[df['여의리(의령군)_수위'] == '-'])
print(df[df['함안군(서출리)_수위'] == '-'])

Empty DataFrame
Columns: [Unnamed: 0, 여의리(의령군)_수위, 의령군(정암교)_수위, 함안군(서출리)_수위, 합천군(적포교)_수위, 길곡_강수량(mm), 대병_강수량(mm), 창녕_강수량(mm), 청덕_강수량(mm), 함안_강수량(mm)]
Index: []
Empty DataFrame
Columns: [Unnamed: 0, 여의리(의령군)_수위, 의령군(정암교)_수위, 함안군(서출리)_수위, 합천군(적포교)_수위, 길곡_강수량(mm), 대병_강수량(mm), 창녕_강수량(mm), 청덕_강수량(mm), 함안_강수량(mm)]
Index: []


# Make Sliding Window

In [35]:
def make_sliding_window(data, lag): # data = dataframe, lag = window size

    data = data.drop(['Unnamed: 0'], axis = 1)
    sliding_lst = [data]

    for i in range(lag):
        sliding_lst.append(data.shift(i+1))
    

    sliding_df = pd.concat(sliding_lst, axis=1)
    
    return sliding_df

In [36]:
sliding_df = make_sliding_window(df, 12)
sliding_df

,여의리(의령군)_수위,의령군(정암교)_수위,함안군(서출리)_수위,합천군(적포교)_수위,길곡_강수량(mm),대병_강수량(mm),창녕_강수량(mm),청덕_강수량(mm),함안_강수량(mm),여의리(의령군)_수위,...,함안_강수량(mm),여의리(의령군)_수위,의령군(정암교)_수위,함안군(서출리)_수위,합천군(적포교)_수위,길곡_강수량(mm),대병_강수량(mm),창녕_강수량(mm),청덕_강수량(mm),함안_강수량(mm)
0,0.57,3.83,4.41,0.63,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.57,3.82,4.4,0.63,0.0,0.0,0.0,0.0,0.0,0.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.57,3.81,4.39,0.62,0.0,0.0,0.0,0.0,0.0,0.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.57,3.79,4.38,0.61,0.0,0.0,0.0,0.0,0.0,0.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.57,3.77,4.37,0.60,0.0,0.0,0.0,0.0,0.0,0.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81930,10.09,0.17,1.82,4.36,0.0,0.0,0.0,0.0,0.0,10.09,...,0.0,10.12,0.18,1.85,4.39,0.0,0.0,0.0,0.0,0.0
81931,10.08,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0,10.09,...,0.0,10.11,0.18,1.85,4.39,0.0,0.0,0.0,0.0,0.0
81932,10.09,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0,10.08,...,0.0,10.11,0.17,1.84,4.39,0.0,0.0,0.0,0.0,0.0
81933,10.1,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0,10.09,...,0.0,10.11,0.17,1.84,4.39,0.0,0.0,0.0,0.0,0.0


In [37]:
sliding_df.isnull().sum()

여의리(의령군)_수위     0
의령군(정암교)_수위     0
함안군(서출리)_수위     0
합천군(적포교)_수위     0
길곡_강수량(mm)      0
               ..
길곡_강수량(mm)     12
대병_강수량(mm)     12
창녕_강수량(mm)     12
청덕_강수량(mm)     12
함안_강수량(mm)     12
Length: 117, dtype: int64

In [38]:
sliding_df = sliding_df.truncate(before=12) # Nan 값이 있는 맨 앞 12개 데이터 자르기
df_tgt = df_tgt.truncate(before=12)

## Split Train / Test

In [40]:
# Train : 2012-08-27 13:00:00 ~ 2020-01-01 00:00:00
# Test  : 2020-01-01 01:00:00 ~ 2021-12-31 23:00:00

print(sliding_df.loc[64392]) # 2020-01-01 00:00:00 기준

여의리(의령군)_수위    -0.7
의령군(정암교)_수위     0.5
함안군(서출리)_수위    1.76
합천군(적포교)_수위    4.25
길곡_강수량(mm)      0.0
               ... 
길곡_강수량(mm)      0.0
대병_강수량(mm)      0.0
창녕_강수량(mm)      0.0
청덕_강수량(mm)      0.0
함안_강수량(mm)      0.0
Name: 64392, Length: 117, dtype: object


In [41]:
train_df = sliding_df.truncate(after=64391)
test_df = sliding_df.truncate(before=64392)

train_label_df = df_tgt.truncate(after=64391)
test_label_df = df_tgt.truncate(before=64392)

# Dataset

In [42]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, target):
    
        self.data = data
        self.target = target

        scaler = MinMaxScaler() # 스케일링
        self.data = pd.DataFrame(scaler.fit_transform(self.data)) 
        self.target = pd.DataFrame(scaler.fit_transform(self.target))

        self.data = torch.tensor(self.data.values).float() # 텐서 변환
        self.target = torch.tensor(self.target.values).float()


    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):

        data = self.data[idx]
        target = self.target[idx]

        return data, target

In [43]:
train_dataset = Dataset(train_df, train_label_df)
test_dataset = Dataset(test_df, test_label_df)

# Train / Val Function

In [ ]:
def train(model, train_loader):
    model.train()
    train_loss_arr = []

    for batch_idx, (data, target) in enumerate(train_loader, start = 1):
        target = target.to(device)
        data = data.to(device)

'1.10.2'